In [13]:
import random
import pandas as pd
import utils.DuckbotExptSetupUtils as exp
import os
import json
import utils.Plate_positions as pp


In [10]:
# USER ACTION REQUIRED

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "sp3484"] # Replace with names for unique duckweed genotypes
media = ["Mock", "Salt"] # Replace with names for unique media
reps  = 3 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "test_data/TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

        

In [8]:
# os.chdir('duckbot/')

print(os.getcwd())
# os.chdir('..')


/home/delangeo/duckbot_git_clone/duckbot


In [11]:
path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
# print(os.getcwd())
# os.chdir('..')
# print(os.getcwd())
print(path)

os.chdir(path)
print(os.getcwd())



with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)
print (df[0:])


/home/delangeo/duckbot_git_clone/duckbot/test_data/TestExptDir
/home/delangeo/duckbot_git_clone/duckbot/test_data/TestExptDir
  genotype media  condition_replicate    plate_well_id    Plate Well
0   Sp7498  Mock                    1  Plate_1_Well_A1  Plate_1   A1
1   Sp7498  Salt                    2  Plate_1_Well_A2  Plate_1   A2
2   Sp7498  Salt                    1  Plate_1_Well_A3  Plate_1   A3
3   Sp7498  Mock                    2  Plate_1_Well_A4  Plate_1   A4


In [21]:
#Add X and Y coordinates
plates_to_set_up = [1] #That is drawn from the number of unique plates in the dataframe. Not too hard. 

def add_well_coords_to_df(plate_num, df):
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(plate_num)
    print(well_coord_list_of_dicts)
#     plate_df = df.loc[df['Plate'] == f'Plate_{plate_num}']
    for index, row in df.iterrows():
        print(row['Well'])
        for well in well_coord_list_of_dicts:
            if row['Plate'] == f'Plate_{plate_num}' and row['Well'] == well['well_id']:
                df.loc[index, 'x'] = well['x']
                df.loc[index, 'y'] = well['y']
    print(df)
#     for i, well in enumerate(well_coord_list_of_dicts):
        
#         m.moveTo(x=well["x"], y=well["y"], z=10)

for p in plates_to_set_up:
    add_well_coords_to_df(p, df)
    
        


[{'well_id': 'A1', 'col': 0, 'row': 0, 'x': 10, 'y': 210}, {'well_id': 'A2', 'col': 1, 'row': 0, 'x': 27, 'y': 210}, {'well_id': 'A3', 'col': 2, 'row': 0, 'x': 44, 'y': 210}, {'well_id': 'A4', 'col': 3, 'row': 0, 'x': 61, 'y': 210}, {'well_id': 'A5', 'col': 4, 'row': 0, 'x': 78, 'y': 210}, {'well_id': 'A6', 'col': 5, 'row': 0, 'x': 95, 'y': 210}, {'well_id': 'B1', 'col': 0, 'row': 0, 'x': 10, 'y': 210}, {'well_id': 'B2', 'col': 1, 'row': 1, 'x': 27, 'y': 193}, {'well_id': 'B3', 'col': 2, 'row': 1, 'x': 44, 'y': 193}, {'well_id': 'B4', 'col': 3, 'row': 1, 'x': 61, 'y': 193}, {'well_id': 'B5', 'col': 4, 'row': 1, 'x': 78, 'y': 193}, {'well_id': 'B6', 'col': 5, 'row': 1, 'x': 95, 'y': 193}, {'well_id': 'C1', 'col': 0, 'row': 2, 'x': 10, 'y': 176}, {'well_id': 'C2', 'col': 1, 'row': 2, 'x': 27, 'y': 176}, {'well_id': 'C3', 'col': 2, 'row': 2, 'x': 44, 'y': 176}, {'well_id': 'C4', 'col': 3, 'row': 2, 'x': 61, 'y': 176}, {'well_id': 'C5', 'col': 4, 'row': 2, 'x': 78, 'y': 176}, {'well_id': '

In [31]:
#Create new dictionary
media_df = df.groupby(['media']) #Returns a list of tuples with [0] being the group key and [1] the dataframe
media_dicts = []
for m in media_df:
    well_coords = []
    for index, row in m[1].iterrows():
        this_well = []
        this_well.append(row['x'])
        this_well.append(row['y'])
        well_coords.append(this_well)
    media_dicts.append({'media': m[0], 'well-coords' : well_coords})
    
print(media_dicts)

[{'media': 'Mock', 'well-coords': [[10.0, 210.0], [61.0, 210.0]]}, {'media': 'Salt', 'well-coords': [[27.0, 210.0], [44.0, 210.0]]}]


In [7]:

bedplate_z_offest = 10
plate_column_offset = 141
plate_row_offset = 97

well_column_offset = 19
well_row_offset = 19

plate_start_position = {"x" : 27, "y" : 267}#The way we have the bedplate set up is basically upside down relative to what the Duetboard is expecting. So instead of calculating our y up from zero we go down from 300. 

plates = [{"plate_id": 1, "col": 0, "row": 1},
          {"plate_id": 2, "col": 0, "row": 2},
          {"plate_id": 3, "col": 1, "row": 0},
          {"plate_id": 4, "col": 1, "row": 1},
          {"plate_id": 5, "col": 1, "row": 2}]



wells_in_plate= [{"well_id" : "A1", "col": 0, "row": 0}                          ,
                 {"well_id" : "A2", "col": 1, "row": 0},
                 {"well_id" : "A3", "col": 2, "row": 0},
                 {"well_id" : "A4", "col": 3, "row": 0},
                 {"well_id" : "A5", "col": 4, "row": 0},
                 {"well_id" : "A6", "col": 5, "row": 0},
                 {"well_id" : "B1", "col": 0, "row": 0},
                 {"well_id" : "B2", "col": 1, "row": 1},
                 {"well_id" : "B3", "col": 2, "row": 1},
                 {"well_id" : "B4", "col": 3, "row": 1},
                 {"well_id" : "B5", "col": 4, "row": 1},
                 {"well_id" : "B6", "col": 5, "row": 1},
                 {"well_id" : "C1", "col": 0, "row": 2},
                 {"well_id" : "C2", "col": 1, "row": 2},
                 {"well_id" : "C3", "col": 2, "row": 2},
                 {"well_id" : "C4", "col": 3, "row": 2},
                 {"well_id" : "C5", "col": 4, "row": 2},
                 {"well_id" : "C6", "col": 5, "row": 2},
                 {"well_id" : "D1", "col": 0, "row": 3},
                 {"well_id" : "D2", "col": 1, "row": 3},
                 {"well_id" : "D3", "col": 2, "row": 3},
                 {"well_id" : "D4", "col": 3, "row": 3},
                 {"well_id" : "D5", "col": 4, "row": 3},
                 {"well_id" : "D6", "col": 5, "row": 3}]



def fetch_plate_wellpostions(plate_num):
    plate_of_interest = next(p for p in plates if p["plate_id"] == plate_num)
    start_x = plate_start_position["x"] + (plate_of_interest["col"] * plate_column_offset)
    start_y = plate_start_position["y"] - (plate_of_interest["row"] * plate_row_offset)
    for i, well in enumerate(wells_in_plate):
        well["x"] = start_x + (well["col"] * well_column_offset)
        well["y"] = start_y - (well["row"] * well_row_offset)
    return(wells_in_plate)


In [8]:
print(plates)

[{'plate_id': 1, 'col': 0, 'row': 1}, {'plate_id': 2, 'col': 0, 'row': 2}, {'plate_id': 3, 'col': 1, 'row': 0}, {'plate_id': 4, 'col': 1, 'row': 1}, {'plate_id': 5, 'col': 1, 'row': 2}]


In [14]:
whole_bedplate_positions = []

for p in plates:
    plate_dict = {}
    start_x = plate_start_position["x"] + (p["col"] * plate_column_offset)
    start_y = plate_start_position["y"] - (p["row"] * plate_row_offset)
    for i, well in enumerate(wells_in_plate):
        well["x"] = start_x + (well["col"] * well_column_offset)
        well["y"] = start_y - (well["row"] * well_row_offset)
    plate_dict["plate"] = p['plate_id']
    plate_dict["well_dict"] = wells_in_plate
    whole_bedplate_positions.append(plate_dict)

print(whole_bedplate_positions)


bedplate_df = pd.DataFrame(whole_bedplate_positions)
print(bedplate_df)

[{'plate': 1, 'well_dict': [{'well_id': 'A1', 'col': 0, 'row': 0, 'x': 168, 'y': 73}, {'well_id': 'A2', 'col': 1, 'row': 0, 'x': 187, 'y': 73}, {'well_id': 'A3', 'col': 2, 'row': 0, 'x': 206, 'y': 73}, {'well_id': 'A4', 'col': 3, 'row': 0, 'x': 225, 'y': 73}, {'well_id': 'A5', 'col': 4, 'row': 0, 'x': 244, 'y': 73}, {'well_id': 'A6', 'col': 5, 'row': 0, 'x': 263, 'y': 73}, {'well_id': 'B1', 'col': 0, 'row': 0, 'x': 168, 'y': 73}, {'well_id': 'B2', 'col': 1, 'row': 1, 'x': 187, 'y': 54}, {'well_id': 'B3', 'col': 2, 'row': 1, 'x': 206, 'y': 54}, {'well_id': 'B4', 'col': 3, 'row': 1, 'x': 225, 'y': 54}, {'well_id': 'B5', 'col': 4, 'row': 1, 'x': 244, 'y': 54}, {'well_id': 'B6', 'col': 5, 'row': 1, 'x': 263, 'y': 54}, {'well_id': 'C1', 'col': 0, 'row': 2, 'x': 168, 'y': 35}, {'well_id': 'C2', 'col': 1, 'row': 2, 'x': 187, 'y': 35}, {'well_id': 'C3', 'col': 2, 'row': 2, 'x': 206, 'y': 35}, {'well_id': 'C4', 'col': 3, 'row': 2, 'x': 225, 'y': 35}, {'well_id': 'C5', 'col': 4, 'row': 2, 'x': 2

In [37]:
def fetch_well_position(plate_num, well_id):
    coords = {}
    for p in whole_bedplate_positions:
        if str(plate_num) == str(p['plate']):
            for w in p["well_dict"]:
                if w['well_id'] == well_id:
                    coords['x'] = w['x']
                    coords['y'] = w['y']
    return(coords)
                    

In [38]:
my_well = fetch_well_position(2, "D1")
print(my_well)
print(my_well['x'])
print(my_well['y'])


{'x': 168, 'y': 16}
168
16
